In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

import rtbm.layers as layers
import rtbm.model as mdl

import warnings
warnings.filterwarnings('ignore')

from rtbm.costfunctions import mse, crossentropy
from rtbm import minimizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelBinarizer

from scipy import signal

RTBM_precision= 1e-5

# Example 1: Sine-Mixture with gaussian noise

In [ ]:
def func(x,n):
    return 0.5*np.sin(1*x+0.1)+0.75*np.sin(0.25*x-0.3)+n*np.random.normal(0,0.2,x.shape[2])

X = np.linspace(0, 100, 500)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None],1).reshape((1,X.shape[1]))

In [ ]:
plt.plot(X.flatten(), Y.flatten(),"ob-")

In [ ]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(1,3,phase=1j))
M.add(layers.DiagExpectationUnitLayer(3,3,phase=1j))
M.add(layers.DiagExpectationUnitLayer(3,1,phase=1j))

In [ ]:
minim = minimizer.CMA(parallel=True)
minim.train(mse, M, X, Y, tolfun=1e-4)

In [ ]:
npoints = 500
test_X = (np.linspace(0, 100, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og-", label='target')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"ob-", label='fit')
plt.legend()

# Example 2: Sine-Cosine-Mixture with trend and gaussian noise

In [ ]:
def func(x,n):
    return 0.02*x+0.5*np.sin(1*x+0.1)+0.75*np.cos(0.25*x-0.3)+n*np.random.normal(0,0.2,x.shape[2])

X = np.linspace(0, 100, 500)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None],1).reshape((1,X.shape[1]))

In [ ]:
Y = func(X[:,None],0).reshape((1,X.shape[1]))

plt.plot(X.flatten(), Y.flatten(),"-")
plt.xlabel("t")
plt.ylabel("y")
plt.savefig("sincosmix.eps", format="eps")
plt.show()


In [ ]:
Y = func(X[:,None],1).reshape((1,X.shape[1]))

plt.plot(X.flatten(), Y.flatten(),"-")
plt.xlabel("t")
plt.ylabel("y")
plt.savefig("sincosmix_noise.eps", format="eps")

plt.show()



In [ ]:
M = mdl.Model()

M.add(layers.DiagExpectationUnitLayer(1,3))
M.add(layers.DiagExpectationUnitLayer(3,3))
M.add(layers.DiagExpectationUnitLayer(3,2))
M.add(layers.DiagExpectationUnitLayer(2,1))

In [ ]:
minim = minimizer.CMA(parallel=True)
minim.train(mse, M, X, Y, tolfun=1e-4)

In [ ]:
npoints = 500
test_X = (np.linspace(0, 100, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og", label='train')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"b-", label='fit', linewidth=5)
plt.legend()
plt.xlabel("t")
plt.ylabel("y")
plt.savefig("sincosmix_10-4.eps", format="eps")
plt.show()

# Example 3: Saw-tooth

In [ ]:
def func(x,n):
    return signal.sawtooth(x)

X = np.linspace(0, 30, 500)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None],1).reshape((1,X.shape[1]))

In [ ]:
plt.plot(X.flatten(), Y.flatten(),"ob-")

In [ ]:
M = mdl.Model()

M.add(layers.DiagExpectationUnitLayer(1,100))
minim = minimizer.CMA(parallel=True)
minim.train(mse, M, X, Y, maxiter=100)

In [ ]:
npoints = 500
test_X = (np.linspace(0, 30, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og-", label='target')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"ob-", label='fit')
plt.xaxis("t")
plt.yaxis("y(t)")
plt.savefig("sawtooth_reg.eps", format="eps")
plt.legend()

# Example 3: MNIST (work in progress...)

In [48]:
# Load MNIST dataset
MNIST_train = pd.read_csv('~/data/mnist_train.csv', delimiter=",",header=None).values
MNIST_test  = pd.read_csv('~/data/mnist_test.csv', delimiter=",",header=None).values

# Prepare data (normalized onto [0,1])
Y_train = MNIST_train[0:10000,0]
X_train = MNIST_train[0:10000,1:]/255.0

Y_test = MNIST_test[:,0]
X_test = MNIST_test[:,1:]/255.0

In [49]:
M = mdl.Model()
#M.add(layers.Linear(784,1,paramBound=100))
M.add(layers.DiagExpectationUnitLayer(784,1,Qmax=5,paramBound=5))
#M.add(layers.DiagExpectationUnitLayer(784,10,Qmax=3,paramBound=5))
#M.add(layers.DiagExpectationUnitLayer(25,10,Qmax=3,paramBound=5))
#M.add(layers.DiagExpectationUnitLayer(10,1))
#M.add(layers.SoftMaxLayer(10))
#M.add(layers.MaxPosLayer(10,1))

#enc = LabelBinarizer()
#enc.fit(np.diag([1,1,1,1,1,1,1,1,1,1]))
#enc.classes_ = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
#T=enc.transform(Y_train)


In [50]:
minim = minimizer.CMA(True)

sol=minim.train(mse, M, np.transpose(X_train), np.transpose(Y_train), maxiter=1000)

CMA on 32 cpu(s) enabled
(12_w,24)-aCMA-ES (mu_w=7.0,w_1=24%) in dimension 786 (seed=332977, Sat Oct 28 23:33:48 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     24 3.740086906930568e+01 1.0e+00 4.95e-01  5e-01  5e-01 0:01.0
    2     48 2.539988884050923e+01 1.0e+00 4.91e-01  5e-01  5e-01 0:01.9
    3     72 1.706350302369885e+01 1.0e+00 4.87e-01  5e-01  5e-01 0:02.7
    7    168 9.036257229987459e+00 1.0e+00 4.73e-01  5e-01  5e-01 0:06.4
   12    288 6.933065752440294e+00 1.0e+00 4.59e-01  5e-01  5e-01 0:11.4
   18    432 6.244749497768543e+00 1.0e+00 4.45e-01  4e-01  4e-01 0:17.1
   25    600 5.384451218800711e+00 1.0e+00 4.33e-01  4e-01  4e-01 0:23.5
   33    792 5.143072398586204e+00 1.0e+00 4.22e-01  4e-01  4e-01 0:31.1
   42   1008 5.042708047820132e+00 1.0e+00 4.12e-01  4e-01  4e-01 0:39.3
   52   1248 4.905101428676931e+00 1.0e+00 4.04e-01  4e-01  4e-01 0:48.6
   64   1536 4.763581933132180e+00 1.0e+00 3.97e-01  4e-01  4e-01 0:59.6
   77 

In [51]:
#P=np.argmax(np.real(M.predict(np.transpose(X_train))),axis=0)
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

             precision    recall  f1-score   support

        0.0       0.57      0.21      0.31      1001
        1.0       0.37      0.30      0.33      1127
        2.0       0.17      0.21      0.19       991
        3.0       0.20      0.25      0.22      1032
        4.0       0.14      0.20      0.16       980
        5.0       0.14      0.23      0.17       863
        6.0       0.19      0.23      0.21      1014
        7.0       0.23      0.22      0.22      1070
        8.0       0.19      0.13      0.16       944
        9.0       0.38      0.13      0.20       978
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0
       12.0       0.00      0.00      0.00         0
       13.0       0.00      0.00      0.00         0

avg / total       0.26      0.21      0.22     10000

[[211 349 236 109  50  28  14   4   0   0   0   0   0   0]
 [ 76 342 391 178  81  37  15   5   1   1   0   0   0   0]
 [ 48 116 211 256 198 100  37  

In [52]:
#P=np.argmax(np.real(M.predict(np.transpose(X_test))),axis=0)
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))


print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

             precision    recall  f1-score   support

        0.0       0.58      0.21      0.30       980
        1.0       0.39      0.33      0.36      1135
        2.0       0.16      0.18      0.17      1032
        3.0       0.19      0.23      0.21      1010
        4.0       0.15      0.20      0.17       982
        5.0       0.16      0.26      0.20       892
        6.0       0.15      0.21      0.18       958
        7.0       0.20      0.20      0.20      1028
        8.0       0.22      0.16      0.18       974
        9.0       0.36      0.13      0.19      1009
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0
       12.0       0.00      0.00      0.00         0
       13.0       0.00      0.00      0.00         0
       14.0       0.00      0.00      0.00         0

avg / total       0.26      0.21      0.22     10000

[[202 331 234 121  54  20  13   4   1   0   0   0   0   0   0]
 [ 73 372 392 159  79  40  13   5